In [ ]:
#! /usr/bin/python3
import os,sys
print("Extract Utility: Using osmium-tool ");

os.system("mason_packages/.link/bin/osmium --version | head -n 1");
print("");

In [ ]:
def get_bbox(path):
    print("Found: ",path)
    with open(path,'r') as file:
        return file.read()

In [ ]:
def extract_bbox(path,name,found=False):
    if (found):
        bbox = get_bbox(path+"/"+name)
        command = "mason_packages/.link/bin/osmium extract --with-history --bbox={0} -o EXTRACTS/{1}.osh.pbf planet-history.latest.osm.pbf".format(bbox,name)
        print("Running command: \n",command)
        
    else:
        for p in os.listdir(path):
            if os.path.isdir(path+"/"+p):
                extract_bbox(path+"/"+p, name)
            elif p==name:
                return extract_bbox(path,name,found=True)

In [ ]:
extract_bbox("bboxes",'detroit')

## US Cities

In [ ]:
import os, json

In [ ]:
cities = json.load(open('us-cities.json','r'))

In [ ]:
for city in cities['features']:
    file = city['properties']['id']
    inputFile = "/home/cc/full-history-utils/EXTRACTS/"+file+".osh.pbf"
    fileRoot  = "/data/EXTRACTS/"+file
    string = "run.sh {0} {1}".format(inputFile, fileRoot)
    print(string)

## Split the Planet by Quadkeys

In [4]:
import numpy as np, json

In [5]:
d2r = np.pi / 180,
r2d = 180 / np.pi

def quadkey_to_tile(quadkey):
    """Transform quadkey to tile coordinates"""
    tile_x, tile_y = (0, 0)
    level = len(quadkey)
    for i in range(level):
        bit = level - i
        mask = 1 << (bit - 1)
        if quadkey[level - bit] == '1':
            tile_x |= mask
        if quadkey[level - bit] == '2':
            tile_y |= mask
        if quadkey[level - bit] == '3':
            tile_x |= mask
            tile_y |= mask
    return [tile_x, tile_y, level]

def tile_to_bbox(tile):
    e = tile2lon(tile[0] + 1, tile[2])
    w = tile2lon(tile[0], tile[2])
    s = tile2lat(tile[1] + 1, tile[2])
    n = tile2lat(tile[1], tile[2])
    return [w, s, e, n]

def tile2lon(x, z) :
    return x / np.power(2, z) * 360 - 180

def tile2lat(y, z) :
    n = np.pi - 2 * np.pi * y / np.power(2, z)
    return r2d * np.arctan(0.5 * (np.exp(n) - np.exp(-n)))

In [7]:
zoomLevel = 2

extracts = []
for i in range(4**zoomLevel):
    out = np.base_repr(i, base=4)
    if len(out) < zoomLevel:
        out = "0"+out
    bbox = tile_to_bbox(quadkey_to_tile(out))
    extracts.append({
        "output": "quadkey_"+out+".osh.pbf",
        "description" : "quadkey_"+out,
        "bbox":bbox
    })
# json.dump({"extracts":extracts}, open('quadkey_extract_config.json','w'),indent=2)

In [34]:
#!/usr/bin/python
zoomLevel = 3

QUADS = "QUAD_EXTRACTS"
BASEDIR  = "QUADS"

def pad0(string):
    string = "0" + string
    if len(string)<zoomLevel:
        return pad0(string)
    return string

extracts = []
for i in range(4**zoomLevel):
    quad = np.base_repr(i, base=4)
    if len(quad) < zoomLevel:
        quad = pad0(quad)
    bbox = tile_to_bbox(quadkey_to_tile(quad))
    extracts.append({
        "output": "quadkey_"+quad+".osh.pbf",
        "description" : "quadkey_"+quad,
        "bbox": ",".join( [str(e) for e in bbox] )
    })
    

for e in extracts:
    print("Going for ", e['output'])
    command = "mason_packages/.link/bin/osmium extract --set-bounds --with-history --overwrite --bbox={0} --verbose \
 -o {3}/{1} {2}".format(e['bbox'],e['output'],'history-latest.osm.pbf',QUADS)
    print(command)
    
    print("\nNow going for the full run!\n")
    
    command = "./run.sh {0}/{1} {2}/{3}".format(QUADS,e['output'],BASEDIR, e['output'][:-8])
    print(command)
    print()

Going for  quadkey_000.osh.pbf
mason_packages/.link/bin/osmium extract --set-bounds --with-history --overwrite --bbox=-180.0,79.17133464081945,-135.0,85.0511287798066 --verbose  -o QUAD_EXTRACTS/quadkey_000.osh.pbf history-latest.osm.pbf

Now going for the full run!

./run.sh QUAD_EXTRACTS/quadkey_000.osh.pbf QUADS/quadkey_000

Going for  quadkey_001.osh.pbf
mason_packages/.link/bin/osmium extract --set-bounds --with-history --overwrite --bbox=-135.0,79.17133464081945,-90.0,85.0511287798066 --verbose  -o QUAD_EXTRACTS/quadkey_001.osh.pbf history-latest.osm.pbf

Now going for the full run!

./run.sh QUAD_EXTRACTS/quadkey_001.osh.pbf QUADS/quadkey_001

Going for  quadkey_002.osh.pbf
mason_packages/.link/bin/osmium extract --set-bounds --with-history --overwrite --bbox=-180.0,66.51326044311186,-135.0,79.17133464081945 --verbose  -o QUAD_EXTRACTS/quadkey_002.osh.pbf history-latest.osm.pbf

Now going for the full run!

./run.sh QUAD_EXTRACTS/quadkey_002.osh.pbf QUADS/quadkey_002

Going for 